<a href="https://colab.research.google.com/github/DiegoRomanCortes/AED/blob/master/Tarea_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Enunciado](https://www.u-cursos.cl/ingenieria/2020/1/CC3001/1/tareas/r/1_CC3001-Tarea2-Diagrama_de_estados.pdf)

In [25]:
dic = {}

def leerlinea(str):
  linea = ''
  for char in str:
    linea += char
  return linea

def asignar(str):
  linea = leerlinea(str)
  nombrevar = ''
  valor = ''
  i = 0
  while i < len(linea):
    if linea[i] == '=':
      valor = '' 
      i += 1
    elif linea[i] == '_' or esInt(linea[i]) or linea[i].isalpha():
      nombrevar += linea[i]
      i += 1
    else:
      valor += linea[i]
      i += 1
  valor = int(valor)
  dic[nombrevar] = valor
  return nombrevar

def esInt(char):
  b = char in [0, 1, 2, 3, 4, 5, 6, 7, 8 ,9]
  return b

def printear(str):
  print(dic[str])

while True:
  print('Escriba un comando. Escriba salir para cerrar el programa')
  a = input()
  if a =='salir': break
  b = asignar(a)
  

Escriba un comando. Escriba salir para cerrar el programa
salir
